# code from 
https://github.com/yanxp/GAN-VAE/blob/master/GAN%26VAE/gan_vae_tensorflow.py

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from tensorflow.examples.tutorials.mnist import input_data

# set the tensorflow verbosity
tf.logging.set_verbosity(tf.logging.DEBUG)


os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"  

In [2]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
mb_size = 64
z_dim = 100
X_dim = mnist.train.images.shape[1] #784
y_dim = mnist.train.labels.shape[1] #10
h_dim = 128
c = 0
lr = 1e-3

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [4]:
#initialize
# =============================== Q(z|X) ======================================

X = tf.placeholder(tf.float32, shape=[None, X_dim])  #784
Z = tf.placeholder(tf.float32, shape=[None, z_dim])  #100

Q_W1 = tf.Variable(xavier_init([X_dim, h_dim]))      # 784 -> 128
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

Q_W2_mu = tf.Variable(xavier_init([h_dim, z_dim]))   # 128 -> 100
Q_b2_mu = tf.Variable(tf.zeros(shape=[z_dim]))

Q_W2_sigma = tf.Variable(xavier_init([h_dim, z_dim]))
Q_b2_sigma = tf.Variable(tf.zeros(shape=[z_dim]))

theta_Q=[Q_W1,Q_W2_mu,Q_W2_sigma,Q_b1,Q_b2_mu,Q_b2_sigma]
def Q(X):
    h = tf.nn.relu(tf.matmul(X, Q_W1) + Q_b1) #h_dim=128
    z_mu = tf.matmul(h, Q_W2_mu) + Q_b2_mu #100
    z_logvar = tf.matmul(h, Q_W2_sigma) + Q_b2_sigma #100
    return z_mu, z_logvar


def sample_z(mu, log_var):
    eps = tf.random_normal(shape=tf.shape(mu))
    return mu + tf.exp(log_var / 2) * eps  #biao zhun


# X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim])) # 784 -> 128
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


# Z = tf.placeholder(tf.float32, shape=[None, 100]) #random noise

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))  # 100 -> 128
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))  # 128 -> 784
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_log_prob,G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [5]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [6]:
z_mu, z_logvar = Q(X)
z_sample = sample_z(z_mu, z_logvar)
Q_sample,_ = generator(z_sample)

_,G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)
Q_fake, Q_logit_fake=discriminator(Q_sample)


# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))


# Alternative losses:
# -------------------
# E[log P(X|z)]
kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)

recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=Q_sample, labels=X), 1)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake,labels=tf.zeros_like(D_logit_fake)))
G_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake,labels=tf.ones_like(D_logit_fake)))

D_loss = D_loss_real +D_loss_fake+G_loss_fake
G_loss = tf.reduce_mean(0.5*recon_loss)-D_loss
Q_loss = tf.reduce_mean(kl_loss+recon_loss)

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)
Q_solver= tf.train.AdamOptimizer().minimize(Q_loss,var_list=theta_Q)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
mb_size = h_dim  #128
Z_dim = 100

session_config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth=True))
with tf.Session(config = session_config) as sess:
    sess.run(tf.global_variables_initializer())

    if not os.path.exists('out/'):
        os.makedirs('out/')

    i = 0

    for it in range(10):   #1000000
        if it % 1 == 0:      #1000
            samples = sess.run(G_sample, feed_dict={Z: np.random.randn(16, z_dim)})  #
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)

        # X_mb, _ = mnist.train.next_batch(mb_size)
        for i in range(1): #15
            X_mb, _ = mnist.train.next_batch(mb_size)
            _, D_loss_curr = sess.run([D_solver,D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
        _, G_loss_curr= sess.run([G_solver,G_loss], feed_dict={X:X_mb,Z: sample_Z(mb_size, Z_dim)})
        _, Q_loss_curr=sess.run([Q_solver,Q_loss],feed_dict={X: X_mb}) 
        if it % 1 == 0: #1000
            print('Iter: {}'.format(it))
            print('D loss: {:.4}'.format(D_loss_curr))
            print('G loss: {:.4}'.format(G_loss_curr))
            print('Q loss: {:.4}'.format(Q_loss_curr))
            # print()

Iter: 0
D loss: 2.077
G loss: 383.6
Q loss: 748.7
Iter: 1
D loss: 1.952
G loss: 351.7
Q loss: 701.1
Iter: 2
D loss: 1.891
G loss: 328.0
Q loss: 655.7
Iter: 3
D loss: 1.738
G loss: 311.0
Q loss: 617.5
Iter: 4
D loss: 1.608
G loss: 297.3
Q loss: 596.1
Iter: 5
D loss: 1.607
G loss: 288.3
Q loss: 575.7
Iter: 6
D loss: 1.587
G loss: 280.6
Q loss: 563.1
Iter: 7
D loss: 1.516
G loss: 270.6
Q loss: 548.0
Iter: 8
D loss: 1.483
G loss: 264.1
Q loss: 533.6
Iter: 9
D loss: 1.508
G loss: 256.2
Q loss: 517.7
